In [1]:
#pragma cling add_include_path("/home/patrick/include_files/")
#pragma cling add_include_path("/home/patrick/include_files/eigen/")
#include <Eigen/Core>
#include <Eigen/Dense>
#include "Team.h"
#include "Game.h"
#include "Simulator.h"
Team::Team(string name, int id)
{
    this->id = id;
    this->name = name;
}

In [2]:
void Team::addGame(Game* newGame)
{
    this->gameCollection.push_back(newGame);
}

In [3]:
std::vector<Game*> Team::getAllGames()
{
    return this->gameCollection;
}

In [4]:
int Team::getId()
{
    return this->id;
}

In [5]:
string Team::getName()
{
    return this->name;
}

In [6]:
int Team::getNumGamesPlayed()
{
    return this->Team::gameCollection.size();
}

In [7]:
string Team::toString()
{
    return name;
}

In [8]:
void Team::addRank(double rank)
{
    this->rank = rank;
}

In [9]:
double Team::getRank() const 
{
    return this->rank;
}

In [10]:
void printVector(std::vector<int> vect)
{
    for(int i = 0; i < vect.size(); i++)
{
    cout << vect.at(i) << '\n';
}
}

In [11]:
void printVector(std::vector<Team> vect)
{
    for(int i = vect.size()-1; i >= vect.size()-64; i--)
{
    cout << vect.at(i).getId() << '\n';
}
}

In [12]:
#define LESS_THAN operator< //workaround for bug in cling
bool LESS_THAN(const Team &team1, const Team &team2)
{
    return team1.getRank() < team2.getRank();
}
#undef LESS_THAN

In [13]:
Game::Game(int day, Team* team1, Team* team2, int score1, int score2)
{
    this->day = day;
    this->teams.push_back(team1);
    this->teams.push_back(team2);
    this->score1 = score1;
    this->score2 = score2;
}

In [14]:
void createTeams(string teamData) {
    string id, name;
    ifstream teams (teamData);
    if(teams.is_open()){
        while (getline(teams, id, ',')){
            getline(teams, name, '\n');
            int idNum = stoi(id);
            Team* tempTeam = new Team(name, idNum);
            teamCollection.push_back(tempTeam);
            teamMap.insert(pair<int, Team*>(idNum, tempTeam));
            numTeams++;
        }
        teams.close();
    }
    else cout << "Unable to open file";
}

In [15]:
void loadGames(string gameData, int home_field_advantage, bool apply_scaling) {
    gameMatrix = Matrix<double, Dynamic, Dynamic>::Zero(numTeams , numTeams);
    scores = VectorXd::Zero(numTeams);

    string days, date, team1, field1, score1, team2, field2, score2;
    fstream games(gameData);
    if(games.is_open()){
        while(getline(games, days, ',')){
            getline(games, date, ',');
            getline(games, team1, ',');
            getline(games, field1, ',');
            getline(games, score1, ',');
            getline(games, team2, ',');
            getline(games, field2, ',');
            getline(games, score2, ' ');
            getline(games, score2, ' ');

            int daysInt = stoi(days);
            int team_1_Id = stoi(team1);
            int team_2_Id = stoi(team2);
            int team_1_score = stoi(score1);
            int team_2_score = stoi(score2);
            Game tempGame(daysInt, teamMap.at(team_1_Id), teamMap.at(team_2_Id), team_1_score, team_2_score);
            teamMap.at(team_1_Id)->addGame(&tempGame);
            teamMap.at(team_2_Id)->addGame(&tempGame);
            numGamesPlayed++;

            /*
             * Edit game score based on flag.
             * flag = 1: 4 is added to home team score
             */
            if(stoi(field1) == 1){
                team_1_score += home_field_advantage;
            }
            if(stoi(field2) == 1){
                team_2_score += home_field_advantage;
            }

            populateMatrix(team_1_Id, team_2_Id, team_1_score, team_2_score, apply_scaling);
            for(int i = 0; i < numTeams; i++){
                gameMatrix.row(numTeams - 1).col(i) << 1;
            }
            scores.row(numTeams - 1) << 0;
        }
    }
    else cout << "Unable to open file";
}

In [16]:
void populateMatrix(int team1Index, int team2Index, int team_1_score, int team_2_score, bool apply_scaling) {
    gameMatrix.row(team1Index - 1 ).col(team2Index - 1) << -1;
    gameMatrix.row(team2Index -1 ).col(team1Index -1) <<  -1;
    gameMatrix.row(team1Index -1 ).col(team1Index -1) <<  teamMap.at(team1Index)->getNumGamesPlayed();
    gameMatrix.row(team2Index -1 ).col(team2Index -1) <<  teamMap.at(team2Index)->getNumGamesPlayed();
    //implementation of scaling if predicted margin is greater then 21
    if(abs(team_1_score - team_2_score) < HIGH_MARGIN || apply_scaling == false) {
        if (team_1_score > team_2_score) {
            int temp = scores.row(team1Index - 1).value();
            scores.row(team1Index - 1) << temp + (team_1_score - team_2_score);
            temp = scores.row(team2Index - 1).value();
            scores.row(team2Index - 1) << temp + (team_2_score - team_1_score);
        } else {
            int temp = scores.row(team2Index - 1).value();
            scores.row(team2Index - 1) << temp + (team_2_score - team_1_score);
            temp = scores.row(team1Index - 1).value();
            scores.row(team1Index - 1) << temp + (team_1_score - team_2_score);
        }
    }
    else{

        if(team_1_score > team_2_score)
        {
            int pre_margin = team_1_score - team_2_score;
            int post_margin = HIGH_MARGIN + (1/HIGH_MARGIN_SCALE) * (pow((pre_margin - 20),HIGH_MARGIN_SCALE)-1);
            int temp = scores.row(team1Index - 1).value();
            scores.row(team1Index - 1) << temp + post_margin;
            temp = scores.row(team2Index - 1).value();
            scores.row(team2Index - 1) << temp - post_margin;
        } else{
            int pre_margin = team_2_score - team_1_score;
            int post_margin = HIGH_MARGIN + (1/HIGH_MARGIN_SCALE) * (pow((pre_margin - 20),HIGH_MARGIN_SCALE)-1);
            int temp = scores.row(team2Index - 1).value();
            scores.row(team2Index - 1) << temp + post_margin;
            temp = scores.row(team1Index - 1).value();
            scores.row(team1Index - 1) << temp - post_margin;
        }

    }
}

In [17]:
void constructCorrectVector(string filename)
{
string id;
fstream rankings(filename);
    if(rankings.is_open()){
        while(getline(rankings,id,'\n')){
            int id_num = stoi(id);
            correct.push_back(id_num);
        }
    }
}

In [18]:
void createLabels()
{
for(int i = resultVector.size()-1; i >= 288; i--)
{
    labels.push_back(resultVector.at(i).getName());
}
}

In [19]:
double constructAxes()
{
x_axis.clear();
y_axis.clear();
int count = 0;
double sum = 0;
bool found = false;
for(int i = 1; i < 65; i++)
{
    x_axis.push_back(i);
}
for(int i = resultVector.size()-1; i >= 0; i--)
{
    found = false;
    //cout << resultVector.at(i).getId() << endl;
    int current_value = resultVector.at(i).getId();
    for(int j = 1; j<65; j++)
    {
        if(current_value == correct.at(j-1))
        {
            found = true;
            y_axis.push_back(j);
            sum += pow(j - x_axis.at(count),2);
            count += 1;
        }
    }
}
    return sum;
}

In [20]:
void run(int home_field_advantage, bool apply_scaling){
    createTeams(string("/home/patrick/Desktop/madness/Data/NCAA Mens BasketBall 2019/NCAABasketballTeams.txt"));
    loadGames(string("/home/patrick/Desktop/madness/Data/NCAA Mens BasketBall 2019/NCAABasketballGames.txt"), home_field_advantage, apply_scaling);
    solutionVector = gameMatrix.lu().solve(scores);

    map<int, Team*>::iterator itr;
    for(itr = teamMap.begin(); itr != teamMap.end(); ++itr){
        itr->second->addRank(solutionVector.row(itr->second->getId() -1).value());
        resultVector.push_back(*(itr->second));
    }

     sort(resultVector.begin(), resultVector.end());
     vector<Team>::iterator itr2;
     for(itr2 = resultVector.end() - 1; itr2 != resultVector.begin() - 1 ; --itr2){
        cout << itr2->getId() << itr2->getName() << endl;
     }
}

In [21]:
#include "xwidgets/xslider.hpp"
xw::slider<int> slider;
slider.min = 0;
slider.max = 15;
slider.value = 0;
slider.description = "Home Field Advantage";
slider.display();

#include "xwidgets/xcheckbox.hpp"
xw::checkbox checkbox;
checkbox.value = false;
checkbox.description = "Scale Large Margins?";
checkbox.indent = false;
checkbox.display();

A Jupyter widget

A Jupyter widget

In [22]:
int home_field_advantage = slider.value;
bool apply_scaling = checkbox.value;
run(home_field_advantage, apply_scaling);

104 Gonzaga
75 Duke
329 Virginia
205 North Carolina
295 Texas Tech
169 Michigan St
114 Houston
137 Kentucky
289 Tennessee
168 Michigan
17 Auburn
330 Virginia Tech
236 Purdue
92 Florida St
195 Nevada
153 LSU
149 Louisville
172 Mississippi St
326 VCU
49 Clemson
192 NC State
133 Kansas
32 Buffalo
348 Wofford
198 New Mexico St
158 Marquette
47 Cincinnati
292 Texas
308 UCF
160 Maryland
127 Iowa St
320 Utah St
222 Oregon
279 St Mary's CA
285 Syracuse
346 Wisconsin
184 Murray St
328 Villanova
89 Florida
337 Washington
164 Memphis
67 Dayton
126 Iowa
287 TCU
143 Lipscomb
134 Kansas St
94 Fresno St
21 Belmont
217 Ohio St
218 Oklahoma
171 Mississippi
20 Baylor
327 Vermont
14 Arkansas
60 Creighton
123 Indiana
11 Arizona St
194 Nebraska
253 San Francisco
54 Colorado
246 S Dakota St
288 Temple
66 Davidson
166 Miami FL
297 Toledo
353 Youngstown St
112 Hofstra
305 UC Irvine
188 N Kentucky
269 South Florida
226 Penn St
142 Liberty
10 Arizona
95 Furman
352 Yale
261 Seton Hall
316 USC
170 Minnesota
4 Ala

In [23]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

In [24]:
constructCorrectVector("/home/patrick/Desktop/madness/Data/NCAA Mens BasketBall 2019/Correct.txt");
createLabels();
double sse = constructAxes();

$$SSE = \sum \limits _{i=1} ^m(Y_i-\hat{Y_i})^2$$

In [28]:
cout << "SSE = " << sse << endl;

SSE = 17658

In [25]:
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
line.x = y_axis;
line.y = y_axis;
auto ax_x = xpl::axis::initialize(sx)
    .label("predicted")
    .finalize();
auto ax_y = xpl::axis::initialize(sy)
    .label("actual")
    .orientation("vertical")
    .side("left")
    .finalize();

In [26]:
#include "xplot/xtooltip.hpp"
xpl::tooltip def_tt,test;
def_tt.fields = std::vector<xtl::xoptional<std::string>>{"x","y"};
def_tt.labels = std::vector<xtl::xoptional<std::string>>{"Predicted Ranking","Actual Ranking"};

In [27]:
auto scatter1 = xpl::scatter::initialize(sx, sy)
   .x(x_axis)
   .y(y_axis)
   .unhovered_style(::xeus::xjson::parse(R"({"opacity": "0.5"})"))
   .tooltip(def_tt)
   .finalize();
scatter1.names = labels;

In [28]:
auto fig1 = xpl::figure::initialize()
    .padding_x(0.1)
    .padding_y(0.025)
    .finalize();
fig1.add_mark(scatter1);
fig1.add_axis(ax_x);
fig1.add_axis(ax_y);
fig1.add_mark(line);
fig1

A Jupyter widget

ERROR: received bad message: No such comm registered: 41df6d5c1c2a4d6ba0599ff105ee852e
Message content: {"comm_id":"41df6d5c1c2a4d6ba0599ff105ee852e","data":{"content":{"data":{"index":29,"name":" Wisconsin","unique_id":" Wisconsin","x":30,"y":9},"event":"hover"},"method":"custom"}}
ERROR: received bad message: No such comm registered: 41df6d5c1c2a4d6ba0599ff105ee852e
Message content: {"comm_id":"41df6d5c1c2a4d6ba0599ff105ee852e","data":{"content":{"data":{"index":17,"name":" Florida St","unique_id":" Florida St","x":18,"y":12},"event":"hover"},"method":"custom"}}
ERROR: received bad message: No such comm registered: 41df6d5c1c2a4d6ba0599ff105ee852e
Message content: {"comm_id":"41df6d5c1c2a4d6ba0599ff105ee852e","data":{"content":{"data":{"index":43,"name":" Florida","unique_id":" Florida","x":44,"y":22},"event":"hover"},"method":"custom"}}


In [29]:
xpl::label label(sx, sy);
label.x = std::vector<double>{28,0};
label.y = std::vector<double>{0,35};
//label.text = std::vector<std::string>{"Predicted Ranking","Actual Ranking"};
//label.colors = std::vector<std::string>{"black", "black"};
label.default_size = 8;
fig1.add_mark(label);